<a href="https://colab.research.google.com/github/Diljitsingh14/AIML/blob/main/SMOL/SFT_smol_train_on_medical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U bitsandbytes datasets trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
# Huggingface login using colab secrets [make sure you will add secret into notebook]
from huggingface_hub import login
from google.colab import userdata


login(token=userdata.get('hf'))

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import torch
from trl import SFTTrainer,SFTConfig,setup_chat_format

model_name = "HuggingFaceTB/SmolLM2-135M"
device = ("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path = model_name)

model , tokenizer = setup_chat_format(model=model,tokenizer=tokenizer)

finetune_name = "SmolLM2-FT-google-fact"
finetune_tags = ['smol-course',"module_1"]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/704 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [4]:
promt = "What is a panic attack?"

message = [{"role":"user","content":promt}]
formatted_prompt = tokenizer.apply_chat_template(message,tokenize=False)

inputs = tokenizer(formatted_prompt,return_tensors="pt")
outputs = model.generate(
                        **inputs,
                         max_new_tokens=100,
                          temperature=0.9,  # Increase randomness
                          top_p=0.85,      # Use nucleus sampling
                          repetition_penalty=1.2,  # Penalize repeated sequences
                          pad_token_id=tokenizer.eos_token_id

                         )
print("Before Training")
print(tokenizer.decode(outputs[0],skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.85` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Before Training
user
What is a panic attack?
A person who has had an episode of anxiety or fear. A woman with this disorder may have: 1) feelings that she will die, and/or be killed;2) feeling like her life might end in death (this can happen to anyone);3) being afraid for herself because the thought makes her feel bad about what could possibly go wrong if it does). This type of experience usually lasts only minutes but sometimes goes on into hours. It's not uncommon for women to describe their symptoms


In [28]:
# prompt: split ds into train and test set

from datasets import load_dataset
ds = load_dataset(path="heliosbrahma/mental_health_chatbot_dataset",name="default")

# Split the dataset into training and testing sets
ds = ds["train"].train_test_split(test_size=0.2, seed=42) # Example: 80% train, 20% test

# Access the training and testing sets
train_dataset = ds["train"]
test_dataset = ds["test"]

print(f"Training set size: {len(train_dataset)}")
print(f"Testing set size: {len(test_dataset)}")

Training set size: 137
Testing set size: 35


In [6]:
# from datasets import load_dataset
# ds = load_dataset(path="heliosbrahma/mental_health_chatbot_dataset",name="default")

(…)-00000-of-00001-01391a60ef5c00d9.parquet:   0%|          | 0.00/102k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/172 [00:00<?, ? examples/s]

In [29]:
# check if the dataset in the write format as chat format need ie "role" and "content" format.

ds['train'][0]['text'].split("<ASSISTANT>:")[1][1:]

"When it comes to discussing your mental health, it's crucial to find someone who is understanding, supportive, and qualified to provide guidance. \n\nA great starting point is to reach out to a mental health professional, such as a psychologist, psychiatrist, or therapist. These experts are trained to help individuals navigate through various mental health challenges, ranging from anxiety and depression to stress and trauma.\n\nIf you're unsure how to find a mental health professional, you can start by asking for recommendations from your primary care physician, friends, or family. \n\nIf you're not quite ready to speak with a professional yet, consider talking to someone you trust maybe a close friend, family member, or a partner. Sharing your feelings with someone who cares about you can offer relief and a sense of validation. Sometimes, merely venting to a compassionate listener can be incredibly therapeutic."

In [30]:
ds

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 137
    })
    test: Dataset({
        features: ['text'],
        num_rows: 35
    })
})

In [31]:
def process_dataset(sample):
    text = sample['text']
    user = text.split("<ASSISTANT>:")[0][9:-1]
    system = text.split("<ASSISTANT>:")[1][1:]
    sample['text'] = [{"role":"user","content":user},{"role":"system","content":system}]
    sample['text'] =  tokenizer.apply_chat_template(sample['text'], tokenize=False)
    return sample

In [24]:
process_dataset(ds['train'][0])

{'text': '<|im_start|>user\nWhat is a panic attack?<|im_end|>\n<|im_start|>system\nPanic attacks come on suddenly and involve intense and often overwhelming fear. They’re accompanied by very challenging physical symptoms, like a racing heartbeat, shortness of breath, or nausea. Unexpected panic attacks occur without an obvious cause. Expected panic attacks are cued by external stressors, like phobias. Panic attacks can happen to anyone, but having more than one may be a sign of panic disorder, a mental health condition characterized by sudden and repeated panic attacks.<|im_end|>\n'}

In [32]:
ds = ds.map(process_dataset)

Map:   0%|          | 0/137 [00:00<?, ? examples/s]

Map:   0%|          | 0/35 [00:00<?, ? examples/s]

In [33]:
sf_config = SFTConfig(
    output_dir = "./sft_output",
    max_steps = 1000,
    per_device_train_batch_size=4,
    learning_rate=.5e-5,
    logging_steps=10,
    save_steps=100,
    evaluation_strategy="steps",
    eval_steps=50,
    # gradient_accumulation_steps=4,
    hub_model_id=finetune_name
)

trainer = SFTTrainer(
    model=model,
    args=sf_config,
    train_dataset=ds['train'],
    eval_dataset=ds['test'],
    # formatting_func=process_dataset,
    tokenizer=tokenizer,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-33-e122948f7d68>:14: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Map:   0%|          | 0/137 [00:00<?, ? examples/s]

Map:   0%|          | 0/35 [00:00<?, ? examples/s]

In [ ]:
trainer.train()
trainer.save_model(f"./{finetune_name}")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss
50,1.828800,1.833543
100,1.545100,1.652403
150,1.481600,1.619492
200,1.420700,1.605840
250,1.524100,1.599555


In [26]:
# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

In [19]:
trained_model = AutoModelForCausalLM.from_pretrained(f"./{finetune_name}")
trained_model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(49152, 576, padding_idx=2)
    (layers): ModuleList(
      (0-29): 30 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=576, out_features=576, bias=False)
          (k_proj): Linear(in_features=576, out_features=192, bias=False)
          (v_proj): Linear(in_features=576, out_features=192, bias=False)
          (o_proj): Linear(in_features=576, out_features=576, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=576, out_features=1536, bias=False)
          (up_proj): Linear(in_features=576, out_features=1536, bias=False)
          (down_proj): Linear(in_features=1536, out_features=576, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((576,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((576,), eps=1e-05)
      )
    )
    (norm

In [25]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load the fine-tuned model and tokenizer
device = "cuda" if torch.cuda.is_available() else "cpu"

trained_model = AutoModelForCausalLM.from_pretrained(f"./{finetune_name}").to(device)
tokenizer = AutoTokenizer.from_pretrained(f"./{finetune_name}")

# Define the input prompt
prompt = "Write a haiku about programming"

# Format the prompt using the tokenizer's existing chat template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Tokenize the formatted prompt
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

# Generate response
outputs = trained_model.generate(
     **inputs,
    max_new_tokens=100,
    temperature=0.9,  # Increase randomness
    top_p=0.85,      # Use nucleus sampling
    repetition_penalty=1.2,  # Penalize repeated sequences
    pad_token_id=tokenizer.eos_token_id
)

# Decode and print the response
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)


user
Write a haiku about programming
What is the difference between an algorithm and a program? What are some examples of algorithms in everyday life, such as sorting or counting numbers. Write your own example using one of these words: "algorithm" (computer), "program", etc., to describe what you think it means for something to be considered an algorithm/a computer program. How does this relate back to our discussion on how we use computers today? Is there anything that can't be done with code without having written programs first? Why


In [22]:
messages = [{"role": "user", "content": "What are some interesting facts about space?"}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)
print("Formatted Prompt:\n", formatted_prompt)


Formatted Prompt:
 <|im_start|>user
What are some interesting facts about space?<|im_end|>



In [23]:
print("Special Tokens:", tokenizer.special_tokens_map)


Special Tokens: {'bos_token': '<|im_start|>', 'eos_token': '<|im_end|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|im_end|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>']}


In [29]:
# Define the input prompt
prompt = "What are some interesting facts about space?"

# Format the prompt using the chat template
formatted_prompt = f"<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant\n"

# Tokenize the input
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

# Generate a response
outputs = trained_model.generate(
    **inputs,
    max_new_tokens=100,  # Number of tokens to generate
    temperature=0.9,     # Increase randomness
    top_p=0.85,          # Nucleus sampling for diversity
    repetition_penalty=1.2,  # Penalize repeated tokens
    pad_token_id=tokenizer.eos_token_id  # Use <|im_end|> as padding
)

# Decode the generated response
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Extract only the assistant's response
response = response.split("<|im_start|>assistant\n")[-1].split("<|im_end|>")[0].strip()

print("Assistant's Response:\n", response)


Assistant's Response:
 user
What are some interesting facts about space?
assistant
How many planets in the solar system have been discovered so far, and what is their average distance from Earth?bobby
Boy: How do you know that Jupiter has 50 moons. What's its diameter?what it looks like with a telescope or binocularsand how big does it look to us on earth?how much bigger than our moonis this planet larger then we think of as beingthe size of Mars but smallerthan Mercury which was thoughtto be only one-third the
